In [42]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep



In [43]:


chrome_options = Options()
prefs = {
    'profile.default_content_setting_values': {
        'images': 2,  # Блокировать изображения
        'javascript': 1,  # Оставить JavaScript включенным (если нужен для загрузки данных)
    }
}
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--blink-settings=imagesEnabled=false')

driver = webdriver.Chrome(options=chrome_options)

The chromedriver version (142.0.7444.61) detected in PATH at C:\ProgramData\chocolatey\bin\chromedriver.exe might not be compatible with the detected chrome version (143.0.7499.41); currently, chromedriver 143.0.7499.42 is recommended for chrome 143.*, so it is advised to delete the driver in PATH and retry


In [44]:
import re
import math
import pandas as pd
import numpy as np



In [45]:

def to_int(x:str) -> int:
    return int(re.sub(r'\D', '', x))

def to_int_sqm(x: str) -> int:
    x = x[:-1] # последний эл. не берем, т.к. там цифра 2 в метрах квадратных
    return int(re.sub(r'\D', '', x))

def to_normAdress(address: str):
    return address


In [46]:
def process_page(url, city_show: str, type_show): # обрабатывает запрос, на выходе массив info с 200 строками
    driver.get(url)
    
    rents_dirty = driver.find_elements(By.CSS_SELECTOR, "[data-test='offers-list-item-param-price']")
    rents_clean = np.array([elem.get_attribute('title') for elem in rents_dirty])

    prices_per_sqm_dirty = driver.find_elements(By.CSS_SELECTOR, "[class='commercial-list-card-price__item _per-sqm']")
    prices_per_sqm_clean = np.array([elem.text for elem in prices_per_sqm_dirty])

    squares_dirty = driver.find_elements(By.CSS_SELECTOR, "[data-test='offers-list-item-param-total-area']")
    # т.к. чередуются значения, а потом пустое значение
    squares_dirty = [squares_dirty[i] for i in range(0, len(squares_dirty), 2)] 
    squares_clean = np.array([elem.text for elem in squares_dirty])

    urls_dirty = driver.find_elements(By.CSS_SELECTOR, "a[_v-0ed5e441]")
    urls_clean = np.array([elem.get_attribute('href') for elem in urls_dirty])


    addresses_dirty = driver.find_elements(By.CSS_SELECTOR, "[class='link-text']")
    # попадает еще тип самого помещения, поэтому берем со второго значения
    addresses_dirty = [addresses_dirty[i] for i in range(1, len(addresses_dirty), 2)]
    addresses_clean = np.array([elem.text for elem in addresses_dirty])

    # извлекаем из блока нужный кусок
    rents = [to_int(el_rent) for el_rent in rents_clean]
    prices_per_sqm = [to_int_sqm(el_price_per_sqm) for el_price_per_sqm in prices_per_sqm_clean]
    square = [to_int_sqm(el_square) for el_square in squares_clean]
    address = [to_normAdress(el_address) for el_address in addresses_clean]
    urls = urls_clean

    count = min(len(rents), len(prices_per_sqm), len(address))
    info = []
    for i in range(count):
        info.append({
            "rent, rub/month": rents[i],
            "full_address": address[i],
            "city": city_show,
            "price_per_sqm, rub/month": prices_per_sqm[i],
            "square, m^2": square[i],
            "type":type_show,
            "url": urls[i]
        })  
    return info

In [47]:

def process_page_with_retry(url, city_show: str, type_show: str, retries: int = 3, delay: float = 3):
    """
    Несколько раз запускает обработку страницы в случае,если страница вернула 0 объявлений (например, не прогрузилась)
    пуммпумпупмпу
    :url: ссылка на страницу N1
    :city_show: город (для записи в DataFrame)
    :type_show: тип (например, 'аренда', 'покупка')
    :retries: сколько раз повторить при 0 результатах
    :param delay: задержка между попытками в секундах
    :return: список объявлений 
    """
    for attempt in range(1, retries + 1):
        info = process_page(url, city_show, type_show)
        count = len(info)

        if count > 0:
            # страница успешно обработана
            if attempt > 1:
                print(f"Успешно на {attempt}-й попытке: {count} объявлений.")
            return info

        # иначе повторяем
        print(f"Попытка {attempt}: страница пуста ({count} объявлений). Повтор через {delay} c...")
        sleep(delay)

    print(f"Страница {url} трижды вернула 0 объявлений. Пропускаю.")
    return []


In [48]:
def go_pages(count_pages: int, type_id: str, city_id: str, type_bool: bool, city_show: str, type_show): # метод, благоря которому мы идем по страницам в определенном типе недвижимости
  data_pages = []
  for p in range(1, count_pages + 1):
    url = f'https://{city_id}.n1.ru/snyat/kommercheskaya/type-{type_id}/?limit=200&page={p}' if type_bool else f'https://{city_id}.n1.ru/snyat/kommercheskaya/purpose-bar-kafe/?limit=200&page={p}'
    info_url = process_page_with_retry(url, city_show, type_show)
    print(f"Страница {p}: {len(info_url)} объявлений, город: {city_show}")
    data_pages.extend(info_url)
  return data_pages

def count_pages(random_url):
    driver.get(random_url)
    sleep(1)
    elements = driver.find_elements(By.CSS_SELECTOR, "span[_v-e5a383be]")
    text_el = None

    # ищем элемент, в тексте которого есть слово "объявлен"
    for el in elements:
        if "объявлен" in el.text:
            text_el = el.text
            break

    if not text_el:
        print(f"Не найден элемент с количеством объявлений для {random_url}")
        return 1  # хотя бы 1 страница

    # извлекаем число
    numbers = re.sub(r"\D", "", text_el)
    count = int(numbers) if numbers else 0

    if count:
        pages = math.ceil(count / 200)
        print(f"Найдено {count} объявлений -> {pages} страниц")
        return pages
    else:
        print(f"Не удалось извлечь число объявлений для {random_url}")
        return 1



In [49]:
def go_types(city_id, city_show):
    types = ['universalnye', 'torgovye-ploschadi', 'bar']
    full_data_city = []

    for type_id in types:
        type_bool = type_id in ['universalnye', 'torgovye-ploschadi']
        type_show = ''
        if type_id == 'universalnye':
            type_show = 'Универсальное помещение'
        elif type_id == 'torgovye-ploschadi':
            type_show = 'Торговая площадь'
        else:
            type_show = 'Помещение под бар/ресторан'
        random_url = (
        f'https://{city_id}.n1.ru/snyat/kommercheskaya/type-{type_id}/?limit=200&page=1'
        if type_bool else
        f'https://{city_id}.n1.ru/snyat/kommercheskaya/purpose-bar-kafe/?limit=200&page=1'
        )
        count = count_pages(random_url)
        cur_data = go_pages(count, type_id, city_id, type_bool, city_show, type_show)
        full_data_city.extend(cur_data)

    return full_data_city


In [50]:
all_cities = {"msk" : "Москва", "spb" : "Санкт-Петербург"}

In [51]:

for city_id in all_cities:
  city_show = all_cities.get(city_id, "Город не найден")
  estate_city = pd.DataFrame(go_types(city_id, city_show))
  estate_city.to_csv(f"data/estate_{city_id}.csv", index=False)

Найдено 4848 объявлений -> 25 страниц
Страница 1: 200 объявлений, город: Москва
Страница 2: 200 объявлений, город: Москва
Страница 3: 200 объявлений, город: Москва
Страница 4: 200 объявлений, город: Москва
Страница 5: 200 объявлений, город: Москва
Страница 6: 200 объявлений, город: Москва
Страница 7: 200 объявлений, город: Москва
Страница 8: 200 объявлений, город: Москва
Страница 9: 200 объявлений, город: Москва
Страница 10: 200 объявлений, город: Москва
Страница 11: 200 объявлений, город: Москва
Страница 12: 200 объявлений, город: Москва
Страница 13: 200 объявлений, город: Москва
Страница 14: 200 объявлений, город: Москва
Страница 15: 200 объявлений, город: Москва
Страница 16: 200 объявлений, город: Москва
Страница 17: 200 объявлений, город: Москва
Страница 18: 200 объявлений, город: Москва
Страница 19: 200 объявлений, город: Москва
Страница 20: 200 объявлений, город: Москва
Страница 21: 200 объявлений, город: Москва
Страница 22: 200 объявлений, город: Москва
Страница 23: 200 объявлен

In [57]:
estate_mskk = pd.read_csv("data/estate_msk.csv")

In [58]:
estate_mskk

,"rent, rub/month",full_address,city,"price_per_sqm, rub/month","square, m^2",type,url
0,1500000,"Пречистенка, 19/11 ст2",Москва,13636,110,Универсальное помещение,https://msk.n1.ru/view/114210381/?open_card_kn
1,1197919,"Таежная, 1",Москва,1917,625,Универсальное помещение,https://msk.n1.ru/view/115999047/?open_card_kn
2,274809,"Дмитровка Б., 32 ст4",Москва,4084,67,Универсальное помещение,https://msk.n1.ru/view/116261376/?open_card_kn
3,386458,"Шумкина, 20 ст1",Москва,1459,265,Универсальное помещение,https://msk.n1.ru/view/116011103/?open_card_kn
4,68125,"Шумкина, 20 ст1",Москва,2084,32,Универсальное помещение,https://msk.n1.ru/view/115777113/?open_card_kn
...,...,...,...,...,...,...,...
7692,1500000,"Краснопролетарская, 36",Москва,3968,378,Помещение под бар/ресторан,https://msk.n1.ru/view/109731158/?open_card_kn
7693,1176000,"Пресненская набережная, 10с",Москва,6461,182,Помещение под бар/ресторан,https://msk.n1.ru/view/111002230/?open_card_kn
7694,2000000,"Пресненская набережная, 10 ст2",Москва,4819,415,Помещение под бар/ресторан,https://msk.n1.ru/view/109683444/?open_card_kn
7695,300003,"Братиславская, 18 корп. 1",Москва,2041,147,Помещение под бар/ресторан,https://msk.n1.ru/view/112349375/?open_card_kn
